## Should contain
  * Whats pattern recog & detect?
  * Why deep learning?
  * Code

In [2]:
# Import all modules
import pylab
from PIL import Image
import numpy as np
import pandas as pd
from lasagne import layers
from lasagne.updates import nesterov_momentum
from theano.tensor.nnet import softmax
from nolearn.lasagne import NeuralNet
from nolearn.lasagne import BatchIterator

In [3]:
# Load train and test set
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [8]:

print "-----------------------------"
print train.head(5)
print "-----------------------------"
print train.count()

   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0       0    ...            0         0         0         0         0   
1       0    ...            0         0         0         0         0   
2       0    ...            0         0         0         0         0   
3       0    ...            0         0         0         0         0   
4       0    ...            0         0         0         0         0   

   pixel779  pixel780  pixel781  pixel782  pixel783  
0         0         0         0         0         

In [9]:
# Normalizing image and setting X and y to train a model
train_labels = train.ix[:, 0].values.astype(np.int32)
train_images = train.ix[:, 1:].values.astype(np.float32)
train_images /= train_images.std(axis = None)
train_images -= train_images.mean()

test_images = test.values.astype(np.float32)
test_images /= test_images.std(axis = None)
test_images -= test_images.mean()

In [14]:
# Setting architecture of NN
net = NeuralNet(
        layers = [
            ('input', layers.InputLayer),
            ('conv1', layers.Conv2DLayer),
            ('pool1', layers.MaxPool2DLayer),
            ('dropout1', layers.DropoutLayer),
            ('conv2', layers.Conv2DLayer),
            ('pool2', layers.MaxPool2DLayer),
            ('dropout2', layers.DropoutLayer),
            ('hidden3', layers.DenseLayer),
            ('output', layers.DenseLayer),
        ],
    
        input_shape = (None, 1, 28, 28),
        conv1_num_filters = 32, conv1_filter_size = (3, 3),
        pool1_pool_size = (2, 2),
        dropout1_p = 0.2,
        conv2_num_filters = 32, conv2_filter_size = (3, 3),
        pool2_pool_size = (2, 2),
        dropout2_p = 0.2,
        hidden3_num_units = 32,
        output_num_units = 10, output_nonlinearity = softmax,
    
        batch_iterator_train = BatchIterator(batch_size = 150),
        batch_iterator_test = BatchIterator(batch_size = 150),
    
        update = nesterov_momentum,
        update_learning_rate = 0.01,
        update_momentum = 0.9,
    
        use_label_encoder = True,
        regression = False,
        max_epochs = 100,
        verbose = 1,
    )

In [16]:
# Reshape dataset to fit to NN
X = train_images.reshape(-1, 1, 28, 28)
y = train_labels

test_x = test_images.reshape(-1, 1, 28, 28)

In [17]:
# Train NN
net.fit(X, y)

KeyboardInterrupt: 

## Notes


  * ### Image Classification[1]
    * Image classification is the task of assigning an input image one label from a fixed set of categories
    * Challenges in Image classification
        * Viewpoint variation
        * Scale variation
        * Deformation
        * Occlusion
        * Illumination conditions
        * Background clutter
        * Intra-class variation
    * To solve this problems, we take a data-driven approach
    * #### Classification pipeline
        * Input : (aka) training data
        * Learning : training a classifier
        * Evaluation : testing our predictions
  
  * ### Neural Network
    * Neural Networks are modeled as collections of neurons that are connected in an acyclic graph
    * Cycles in NN are not allowed
    * Activation functions of a NN
        1. Sigmoid : $$\sigma(x) = 1 / (1 + e^{-x})$$
        2. Tanh : $$\tanh(x)$$
        3. ReLU : $$\max(0, x)$$
        4. Leaky ReLU : $$1(x < 0)(\alpha x) + 1(x >= 0)(x)$$
        5. Maxout : $$\max(\omega_1 ^T + b_1, \omega_2 ^T x + b_2)$$
        
        
  * ### Convolutional Neural Network
    * A ConvNet is a list of layers that transform the image volume into an output volume through a differentable function
    * Types of layers
        * Convolutional Layer
        * Pooling Layer
        * Normalization layer
        * Fully-connected layer
        
  * References
      1. http://cs231n.github.io/classification/ , http://cs231n.github.io/neural-networks-1/ , http://cs231n.github.io/convolutional-networks/ 
      2. [A few useful things to know about machine learning](http://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)
      3. [Recognizing and learning object categories](http://people.csail.mit.edu/torralba/shortCourseRLOC/index.html)